In [1]:
import json
import collections

In [2]:
graph_prototype = {'version': 'gmxapi_graph_0_2',
                   'elements': {}}
element_prototype = {'label': None,
                     'namespace': None,
                     'operation': None,
                     'input': {},
                     'output': {},
                     'depends': [],
                     'interface': {}}

In [ ]:
# Key types

class ObjectName(collections.UserString):
    def __init__(self, name: str):
        # TODO: validation
        super().__init__(name)

class Label(collections.UserString):
    def __init__(self, name: str):
        # TODO: validation
        super().__init__(name)

# Value types

class Data:
    """Literal data for work graph input values.
    
    Literal data objects are dense arrays of uniform element type. 
    """

class Collection:
    """Value that is a nested mapping of keys to value objects.
    
    Keys are strings subject to the constraints of Label.
    
    Values are Data, Collections, or References.
    """

class Reference:
    """Work record value object that is a reference to another graph entity."""

In [ ]:
# Manage the structure and rules of a work record.
import copy
class WorkRecord(collections.UserDict):
    def __init__(self):
        super().__init__(graph_prototype)
        
    def __getitem__(self, item):
        return copy.deepcopy(super().__getitem__(item))

    def as_json(self, **kwargs):
        import json
        json.dumps(self.data, **kwargs)
        

In [4]:
record = WorkRecord()
print(str(record))
print(repr(record))
print(record)